In [2]:
struct NNKolmogorov{C,O} <: NeuralNetDiffEqAlgorithm
    chain::C
    opt::O
    
end
NNKolmogorov(chain ; opt=Flux.ADAM(0.1)) = NNKolmogorov(chain , opt)

function DiffEqBase.solve(
    prob::KolmogorovPDEProblem,
    alg::NeuralNetDiffEqAlgorithm,
    args...;
    dt,
    timeseries_errors = true,
    save_everystep=true,
    adaptive=false,
    abstol = 1f-6,
    verbose = false,
    maxiters = 100)

    DiffEqBase.isinplace(prob) && error("Only out-of-place methods are allowed!")

    tspan = prob.tspan
    phi = prob.phi
    xspan = prob.xspan
    T = tspan[2]

    #hidden layer
    chain  = alg.chain
    opt    = alg.opt
    ps     = Flux.params(chain)
    xi     = randn(Uniform(a , b))
    x_sde = pdf.(N , xi)
    y = phi(x_sde)
    data   = Iterators.repeated((x , y), maxiters)
    
    #MSE Loss Function
    loss(x , y) =Flux.mse(chain(x), y)

    cb = function ()
        l = loss()
        verbose && println("Current loss is: $l")
        l < abstol && Flux.stop()
    end
        Flux.train!(loss, ps, data, opt; cb = cb)
        
        chain(x_sde)

   
end #solve


UndefVarError: UndefVarError: NeuralNetDiffEqAlgorithm not defined

In [9]:



struct KolmogorovPDEProblem{Phi , X , T } <: DiffEqBase.DEProblem
    phi::Phi
    xspan::Tuple{X,X}
    tspan::Tuple{T,T}
    KolmogorovPDEProblem(phi , xspan , tspan) = new{typeof(phi),eltype(tspan),eltype(xspan)}(
                                                         phi , xspan , tspan)
end


LoadError: UndefVarError: @reexport not defined

In [4]:
Base.summary(prob::KolmogorovPDEProblem) = string(nameof(typeof(prob)))
function Base.show(io::IO, A::KolmogorovPDEProblem)
  println(io,summary(A))
  print(io,"timespan: ")
  show(io,A.tspan)
  print(io,"xspan: ")
  show(io,A.xspan)
end

UndefVarError: UndefVarError: KolmogorovPDEProblem not defined

In [10]:
function phi(m , n)
    return m + n
end

phi (generic function with 1 method)

In [13]:
using DifferentialEquations
dt1 = 0.1/20f0
t1 = 0:dt1:1.0
brownian_values = cumsum([0;[sqrt(dt1)*randn() for i in 1:length(t1)-2]])
W = NoiseGrid(t1,brownian_values)
print()

In [14]:
u = [phi(m , n) for (m ,n) in zip(t1 , W.u)]

DimensionMismatch: DimensionMismatch("dimensions must match")

In [29]:
u = Float64[]
for (m ,n) in zip(t1 , W.u)
    u = vcat(u , [phi(m , n)])
end

In [30]:
u 

200-element Array{Float64,1}:
  0.0                  
 -0.15964992606818035  
 -0.12136868131724722  
 -0.13897630303641761  
 -0.18109792465190871  
 -0.18721198769216751  
 -0.17415683485667535  
 -0.20757187287448323  
 -0.18679178321357734  
 -0.08208849477464895  
 -0.08610062888151902  
 -0.08213043642489859  
 -0.05284923536561381  
  ⋮                    
 -0.1143011667385756   
 -0.12414062560090533  
 -0.29422767410378303  
 -0.43686758203114817  
 -0.35815066391279027  
 -0.27626079671617443  
 -0.19645686083237024  
 -0.14181201971548518  
 -0.012178706172993214 
 -0.061727000889968076 
  0.039923791347428184 
  0.0005565718599399583